# Fuzzy Control

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from typing import Tuple
from matplotlib import patches
from matplotlib.animation import ArtistAnimation
from IPython import display

## Control Target : Vehicle 
- Longitudinal dynamics : Point Mass Model
- Lateral dynamics : Kinematic Bicycle Model

In [ ]:
class Vehicle():
    def __init__(
            self,
            l_f: float = 1.5, # [m]
            l_r: float = 1.0, # [m]
            max_steer_abs: float = 0.523, # [rad]
            max_accel_abs: float = 2.000, # [m/s^2]
            ref_path: np.ndarray = np.array([[-30.0, 0.0], [30.0, 0.0]]),
            delta_t: float = 0.05, # [s]
            visualize: bool = True,
        ) -> None:
        """initialize vehicle environment
        state variables:
            x: x-axis position in the global frame [m]
            y: y-axis position in the global frame [m]
            yaw: orientation in the global frame [rad]
            v: longitudinal velocity [m/s]
        control input:
            steer: front tire angle of the vehicle [rad] (positive in the counterclockwize direction)
            accel: longitudinal acceleration of the vehicle [m/s^2] (positive in the forward direction)
        Note: dynamics of the vehicle is the Kinematic Bicycle Model. 
        """
        # vehicle parameters
        self.l_f = l_f # [m]
        self.l_r = l_r # [m]
        self.wheel_base = l_f + l_r # [m]
        self.max_steer_abs = max_steer_abs # [rad]
        self.max_accel_abs = max_accel_abs # [m/s^2]
        self.delta_t = delta_t #[s]
        self.ref_path = ref_path

        # visualization settings
        self.vehicle_w = 3.00
        self.vehicle_l = 4.00
        self.view_x_lim_min, self.view_x_lim_max = -20.0, 20.0
        self.view_y_lim_min, self.view_y_lim_max = -25.0, 25.0

        # reset environment
        self.visualize_flag = visualize
        self.reset()

    def reset(
            self, 
            init_state: np.ndarray = np.array([0.0, 0.0, 0.0, 0.0]), # [x, y, yaw, v]
        ) -> None:
        """reset environment to initial state"""

        # reset state variables
        self.state = init_state

        # clear animation frames
        self.frames = []

        if self.visualize_flag:
            # prepare figure
            self.fig = plt.figure(figsize=(9,9))
            self.main_ax = plt.subplot2grid((3,4), (0,0), rowspan=3, colspan=3)
            self.minimap_ax = plt.subplot2grid((3,4), (0,3))
            self.steer_ax = plt.subplot2grid((3,4), (1,3))
            self.accel_ax = plt.subplot2grid((3,4), (2,3))

            # graph layout settings
            ## main view
            self.main_ax.set_aspect('equal')
            self.main_ax.set_xlim(self.view_x_lim_min, self.view_x_lim_max)
            self.main_ax.set_ylim(self.view_y_lim_min, self.view_y_lim_max)
            self.main_ax.tick_params(labelbottom=False, labelleft=False, labelright=False, labeltop=False)
            self.main_ax.tick_params(bottom=False, left=False, right=False, top=False)
            ## mini map
            self.minimap_ax.set_aspect('equal')
            self.minimap_ax.axis('off')
            ## steering angle view
            self.steer_ax.set_title("Steering Angle", fontsize="12")
            self.steer_ax.axis('off')
            ## acceleration view
            self.accel_ax.set_title("Acceleration", fontsize="12")
            self.accel_ax.axis('off')
            
            # apply tight layout
            self.fig.tight_layout()

    def update(
            self, 
            u: np.ndarray, 
            delta_t: float = 0.0, 
            append_frame: bool = True, 
            vehicle_traj: np.ndarray = np.empty(0), # vehicle trajectory
        ) -> None:
        """update state variables"""
        # keep previous states
        x, y, yaw, v = self.state

        # prepare params
        l = self.wheel_base
        l_r = self.l_r
        dt = self.delta_t if delta_t == 0.0 else delta_t

        # limit control inputs
        steer = np.clip(u[0], -self.max_steer_abs, self.max_steer_abs)
        accel = np.clip(u[1], -self.max_accel_abs, self.max_accel_abs)

        """< CORE OF VEHICLE DYNAMICS >"""
        # update state variables
        beta = np.arctan(l_r / l * np.tan(steer))
        new_x = x + v * np.cos(yaw + beta) * dt
        new_y = y + v * np.sin(yaw + beta) * dt
        new_yaw = yaw + v / l * np.sin(beta) * dt
        new_v = v + accel * dt
        self.state = np.array([new_x, new_y, new_yaw, new_v])
        """< CORE OF VEHICLE DYNAMICS >"""

        # record frame
        if append_frame:
            self.append_frame(steer, accel, vehicle_traj)

    def get_state(self) -> np.ndarray:
        """return state variables"""
        return self.state.copy()

    def append_frame(self, steer: float, accel: float, vehicle_traj: np.ndarray) -> list:
        """draw a frame of the animation."""
        # get current states
        x, y, yaw, v = self.state

        ### main view ###
        # draw the vehicle shape
        vw, vl = self.vehicle_w, self.vehicle_l
        vehicle_shape_x = [-0.5*vl, -0.5*vl, +0.5*vl, +0.5*vl, -0.5*vl, -0.5*vl]
        vehicle_shape_y = [0.0, +0.5*vw, +0.5*vw, -0.5*vw, -0.5*vw, 0.0]
        rotated_vehicle_shape_x, rotated_vehicle_shape_y = \
            self._affine_transform(vehicle_shape_x, vehicle_shape_y, yaw, [0, 0]) # make the vehicle be at the center of the figure
        frame = self.main_ax.plot(rotated_vehicle_shape_x, rotated_vehicle_shape_y, color='black', linewidth=2.0, zorder=3)

        # draw wheels
        ww, wl = 0.4, 0.7 #[m]
        wheel_shape_x = np.array([-0.5*wl, -0.5*wl, +0.5*wl, +0.5*wl, -0.5*wl, -0.5*wl])
        wheel_shape_y = np.array([0.0, +0.5*ww, +0.5*ww, -0.5*ww, -0.5*ww, 0.0])

        ## rear-left wheel
        wheel_shape_rl_x, wheel_shape_rl_y = \
            self._affine_transform(wheel_shape_x, wheel_shape_y, 0.0, [-0.3*vl, 0.3*vw])
        wheel_rl_x, wheel_rl_y = \
            self._affine_transform(wheel_shape_rl_x, wheel_shape_rl_y, yaw, [0, 0])
        frame += self.main_ax.fill(wheel_rl_x, wheel_rl_y, color='black', zorder=3)

        ## rear-right wheel
        wheel_shape_rr_x, wheel_shape_rr_y = \
            self._affine_transform(wheel_shape_x, wheel_shape_y, 0.0, [-0.3*vl, -0.3*vw])
        wheel_rr_x, wheel_rr_y = \
            self._affine_transform(wheel_shape_rr_x, wheel_shape_rr_y, yaw, [0, 0])
        frame += self.main_ax.fill(wheel_rr_x, wheel_rr_y, color='black', zorder=3)

        ## front-left wheel
        wheel_shape_fl_x, wheel_shape_fl_y = \
            self._affine_transform(wheel_shape_x, wheel_shape_y, steer, [0.3*vl, 0.3*vw])
        wheel_fl_x, wheel_fl_y = \
            self._affine_transform(wheel_shape_fl_x, wheel_shape_fl_y, yaw, [0, 0])
        frame += self.main_ax.fill(wheel_fl_x, wheel_fl_y, color='black', zorder=3)

        ## front-right wheel
        wheel_shape_fr_x, wheel_shape_fr_y = \
            self._affine_transform(wheel_shape_x, wheel_shape_y, steer, [0.3*vl, -0.3*vw])
        wheel_fr_x, wheel_fr_y = \
            self._affine_transform(wheel_shape_fr_x, wheel_shape_fr_y, yaw, [0, 0])
        frame += self.main_ax.fill(wheel_fr_x, wheel_fr_y, color='black', zorder=3)

        # draw the vehicle center circle
        vehicle_center = patches.Circle([0, 0], radius=vw/20.0, fc='white', ec='black', linewidth=2.0, zorder=4)
        frame += [self.main_ax.add_artist(vehicle_center)]

        # draw the reference path
        ref_path_x = self.ref_path[:, 0] - np.full(self.ref_path.shape[0], x)
        ref_path_y = self.ref_path[:, 1] - np.full(self.ref_path.shape[0], y)
        frame += self.main_ax.plot(ref_path_x, ref_path_y, color='black', linestyle="dashed", linewidth=1.5)

        # draw the information text
        text = "vehicle velocity = {v:>+6.1f} [m/s]".format(pos_e=x, head_e=np.rad2deg(yaw), v=v)
        frame += [self.main_ax.text(0.5, 0.02, text, ha='center', transform=self.main_ax.transAxes, fontsize=14, fontfamily='monospace')]

        # draw vehicle trajectory
        if vehicle_traj.any():
            vehicle_traj_x_offset = np.append(np.ravel(vehicle_traj[:, 0]) - np.full(vehicle_traj.shape[0], x), [0.0])
            vehicle_traj_y_offset = np.append(np.ravel(vehicle_traj[:, 1]) - np.full(vehicle_traj.shape[0], y), [0.0])
            frame += self.main_ax.plot(vehicle_traj_x_offset, vehicle_traj_y_offset, color='purple', linestyle="solid", linewidth=2.0)

        ### mini map view ###
        frame += self.minimap_ax.plot(self.ref_path[:, 0], self.ref_path[:,1], color='black', linestyle='dashed')
        rotated_vehicle_shape_x_minimap, rotated_vehicle_shape_y_minimap = \
            self._affine_transform(vehicle_shape_x, vehicle_shape_y, yaw, [x, y]) # make the vehicle be at the center of the figure
        frame += self.minimap_ax.plot(rotated_vehicle_shape_x_minimap, rotated_vehicle_shape_y_minimap, color='black', linewidth=2.0, zorder=3)
        frame += self.minimap_ax.fill(rotated_vehicle_shape_x_minimap, rotated_vehicle_shape_y_minimap, color='white', zorder=2)
        if vehicle_traj.any():
            frame += self.minimap_ax.plot(vehicle_traj[:, 0], vehicle_traj[:, 1], color='purple', linestyle="solid", linewidth=1.0)

        ### control input view ###
        # steering angle
        MAX_STEER = self.max_steer_abs
        PIE_RATE = 3.0/4.0
        PIE_STARTANGLE = 225 # [deg]
        s_abs = np.abs(steer)
        if steer < 0.0: # when turning right
            steer_pie_obj, _ = self.steer_ax.pie([MAX_STEER*PIE_RATE, s_abs*PIE_RATE, (MAX_STEER-s_abs)*PIE_RATE, 2*MAX_STEER*(1-PIE_RATE)], startangle=PIE_STARTANGLE, counterclock=False, colors=["lightgray", "black", "lightgray", "white"], wedgeprops={'linewidth': 0, "edgecolor":"white", "width":0.4})
        else: # when turning left
            steer_pie_obj, _ = self.steer_ax.pie([(MAX_STEER-s_abs)*PIE_RATE, s_abs*PIE_RATE, MAX_STEER*PIE_RATE, 2*MAX_STEER*(1-PIE_RATE)], startangle=PIE_STARTANGLE, counterclock=False, colors=["lightgray", "black", "lightgray", "white"], wedgeprops={'linewidth': 0, "edgecolor":"white", "width":0.4})
        frame += steer_pie_obj
        frame += [self.steer_ax.text(0, -1, f"{np.rad2deg(steer):+.2f} " + r"$ \rm{[deg]}$", size = 14, horizontalalignment='center', verticalalignment='center', fontfamily='monospace')]

        # acceleration
        MAX_ACCEL = self.max_accel_abs
        PIE_RATE = 3.0/4.0
        PIE_STARTANGLE = 225 # [deg]
        a_abs = np.abs(accel)
        if accel > 0.0:
            accel_pie_obj, _ = self.accel_ax.pie([MAX_ACCEL*PIE_RATE, a_abs*PIE_RATE, (MAX_ACCEL-a_abs)*PIE_RATE, 2*MAX_ACCEL*(1-PIE_RATE)], startangle=PIE_STARTANGLE, counterclock=False, colors=["lightgray", "black", "lightgray", "white"], wedgeprops={'linewidth': 0, "edgecolor":"white", "width":0.4})
        else:
            accel_pie_obj, _ = self.accel_ax.pie([(MAX_ACCEL-a_abs)*PIE_RATE, a_abs*PIE_RATE, MAX_ACCEL*PIE_RATE, 2*MAX_ACCEL*(1-PIE_RATE)], startangle=PIE_STARTANGLE, counterclock=False, colors=["lightgray", "black", "lightgray", "white"], wedgeprops={'linewidth': 0, "edgecolor":"white", "width":0.4})
        frame += accel_pie_obj
        frame += [self.accel_ax.text(0, -1, f"{accel:+.2f} " + r"$ \rm{[m/s^2]}$", size = 14, horizontalalignment='center', verticalalignment='center', fontfamily='monospace')]

        # append frame
        self.frames.append(frame)

    # rotate shape and return location on the x-y plane.
    def _affine_transform(self, xlist: list, ylist: list, angle: float, translation: list=[0.0, 0.0]) -> Tuple[list, list]:
        transformed_x = []
        transformed_y = []
        if len(xlist) != len(ylist):
            print("[ERROR] xlist and ylist must have the same size.")
            raise AttributeError

        for i, xval in enumerate(xlist):
            transformed_x.append((xlist[i])*np.cos(angle)-(ylist[i])*np.sin(angle)+translation[0])
            transformed_y.append((xlist[i])*np.sin(angle)+(ylist[i])*np.cos(angle)+translation[1])
        transformed_x.append(transformed_x[0])
        transformed_y.append(transformed_y[0])
        return transformed_x, transformed_y

    def show_animation(self, interval_ms: int) -> None:
        """show animation of the recorded frames"""
        ani = ArtistAnimation(self.fig, self.frames, interval=interval_ms) # blit=True
        html = display.HTML(ani.to_jshtml())
        display.display(html)
        plt.close()

    def save_animation(self, filename: str, interval: int, movie_writer: str="ffmpeg") -> None:
        """save animation of the recorded frames (ffmpeg required)"""
        ani = ArtistAnimation(self.fig, self.frames, interval=interval)
        ani.save(filename, writer=movie_writer)


## Longitudinal Controller : Fuzzy Controller

In [ ]:
class FuzzyLongitudinalController():
    def __init__(
            self,
            slow_thresh_rate: float = +0.6, # [%]
            fast_thresh_rate: float = +1.4, # [%]
            target_velocity: float = 3.0, # [m/s]
    ) -> None:
        """initialize fuzzy controller for keeping constant velocity"""
        # fuzzy control parameters
        self.target_vel = target_velocity
        self.slow_thresh_vel = slow_thresh_rate * target_velocity
        self.fast_thresh_vel = fast_thresh_rate * target_velocity
        self.acc_when_vel_is_slow = +2.0 # [m/s^2]
        self.acc_when_vel_is_mid = 0.0 # [m/s^2]
        self.acc_when_vel_is_fast = -2.0 # [m/s^2]

        self.pre_e = 0.0 # previous tracking error
        self.integrated_e = 0.0 # integrated tracking error

    def calc_control_input(self, observed_vel: float, delta_t: float) -> None:
        """calculate control input"""
        # calculate tracking error, its integral and derivative
        v = observed_vel

        # calculate control input with fuzzy control
        acc_cmd = self.defuzzify(v)

        return acc_cmd

    def defuzzify(self, vel: float) -> float:
        """defuzzigy control input"""
        w_slow = 1.0
        w_mid  = 1.0
        w_fast = 1.0
        acc_cmd = self.acc_when_vel_is_slow * w_slow * self.speed_slow(vel) \
                + self.acc_when_vel_is_mid  * w_mid  * self.speed_mid(vel) \
                + self.acc_when_vel_is_fast * w_fast * self.speed_fast(vel)

        return acc_cmd

    def speed_slow(self, vel: float):
        """fuzzy membership function for slow speed"""
        if vel < self.slow_thresh_vel:
            return 1.0
        elif vel < self.fast_thresh_vel:
            return (self.fast_thresh_vel - vel) / (self.fast_thresh_vel - self.slow_thresh_vel)
        else:
            return 0.0

    def speed_mid(self, vel: float):
        """fuzzy membership function for middle speed"""
        if vel < self.slow_thresh_vel:
            return 0.0
        elif vel < self.target_vel:
            return (vel - self.slow_thresh_vel) / (self.target_vel - self.slow_thresh_vel)
        elif vel < self.fast_thresh_vel:
            return (self.fast_thresh_vel - vel) / (self.fast_thresh_vel - self.target_vel)
        else:
            return 0.0

    def speed_fast(self, vel: float):
        """fuzzy membership function for fast speed"""
        if vel < self.slow_thresh_vel:
            return 0.0
        elif vel < self.fast_thresh_vel:
            return (vel - self.slow_thresh_vel) / (self.fast_thresh_vel - self.slow_thresh_vel)
        else:
            return 1.0


## Lateral Controller : Fuzzy Controller

In [ ]:
class FuzzyLateralController():
    def __init__(
            self,
            ref_path: np.ndarray = np.array([[0.0, 0.0, 0.0, 1.0], [10.0, 0.0, 0.0, 1.0]]),
    ) -> None:
        """initialize fuzzy controller for path-tracking"""
        # fuzzy control parameters
        self.y_e_n_thresh = -1.0 # [m]
        self.y_e_z_thresh = 0.0 # [m]
        self.y_e_p_thresh = +1.0 # [m]
        self.steer_when_y_e_is_n = +0.523 # [rad]
        self.steer_when_y_e_is_z = 0.0 # [rad]
        self.steer_when_y_e_is_p = -0.523 # [rad]

        self.theta_e_n_thresh = -0.523/7.0 # [rad]
        self.theta_e_z_thresh = 0.0 # [rad]
        self.theta_e_p_thresh = +0.523/7.0 # [rad]
        self.steer_when_theta_e_is_n = +0.523 # [rad]
        self.steer_when_theta_e_is_z = 0.0 # [rad]
        self.steer_when_theta_e_is_p = -0.523 # [rad]

        # ref_path info
        self.ref_path = ref_path
        self.prev_waypoints_idx = 0

    def calc_control_input(self, observed_x: np.ndarray, delta_t: float) -> float:
        """calculate control input"""

        # set vehicle state variables from observation
        x = observed_x[0]
        y = observed_x[1]
        yaw = observed_x[2]
        v = observed_x[3]

        # get the waypoint closest to current vehicle position
        _, ref_x, ref_y, ref_yaw, _ = self._get_nearest_waypoint(x, y, update_prev_idx=True)
        if self.prev_waypoints_idx >= self.ref_path.shape[0]-1:
            print("[ERROR] Reached the end of the reference path.")
            raise IndexError

        # which side of the reference path is the car on, the right or the left?
        ## algorithm : http://www.hptown.com/ucad/Ufb00009.htm
        x1, y1 = ref_x, ref_y
        x2, y2 = ref_x + 1.0 * np.cos(ref_yaw), ref_y + 1.0 * np.sin(ref_yaw)
        vx, vy = x2 - x1, y2 - y1
        wx, wy =  x - x1,  y - y1
        s = vx * wy - vy * wx # s>0 : vehicle is on the left of the path, s<0 : vehicle is on the left of the path,

        # get tracking error
        y_e = np.sign(s) * np.sqrt((ref_x-x)**2 + (ref_y-y)**2) # lateral error 
        theta_e = yaw - ref_yaw # heading error
        theta_e = np.arctan2(np.sin(theta_e), np.cos(theta_e)) # normalize heading error to [-pi, pi]

        # calculate control input
        steer_cmd = self.defuzzify(y_e, theta_e)
        return steer_cmd

    def defuzzify(self, y_e: float, theta_e: float) -> float:
        """defuzzigy control input"""
        w_p = 0.6
        w_z = 0.6
        w_n = 0.6

        steer_cmd_y_e = self.steer_when_y_e_is_p * w_p * self.y_e_p(y_e) \
                      + self.steer_when_y_e_is_z * w_z * self.y_e_z(y_e) \
                      + self.steer_when_y_e_is_n * w_n * self.y_e_n(y_e)

        steer_cmd_theta_e = self.steer_when_theta_e_is_p * (1 - w_p) * self.theta_e_p(theta_e) \
                          + self.steer_when_theta_e_is_z * (1 - w_z) * self.theta_e_z(theta_e) \
                          + self.steer_when_theta_e_is_n * (1 - w_n) * self.theta_e_n(theta_e)

        steer_cmd = steer_cmd_y_e + steer_cmd_theta_e
        return steer_cmd

    def y_e_n(self, y_e: float):
        """fuzzy membership function for y_e negative"""
        if y_e < self.y_e_n_thresh:
            return 1.0
        elif y_e < self.y_e_z_thresh:
            return (self.y_e_z_thresh - y_e) / (self.y_e_z_thresh - self.y_e_n_thresh)
        else:
            return 0.0

    def y_e_z(self, y_e: float):
        """fuzzy membership function for y_e zero"""
        if y_e < self.y_e_n_thresh:
            return 0.0
        elif y_e < self.y_e_z_thresh:
            return (y_e - self.y_e_n_thresh) / (self.y_e_z_thresh - self.y_e_n_thresh)
        elif y_e < self.y_e_p_thresh:
            return (self.y_e_p_thresh - y_e) / (self.y_e_p_thresh - self.y_e_z_thresh)
        else:
            return 0.0

    def y_e_p(self, y_e: float):
        """fuzzy membership function for y_e positive"""
        if y_e < self.y_e_z_thresh:
            return 0.0
        elif y_e < self.y_e_p_thresh:
            return (y_e - self.y_e_z_thresh) / (self.y_e_p_thresh - self.y_e_z_thresh)
        else:
            return 1.0

    def theta_e_n(self, theta_e: float):
        """fuzzy membership function for theta_e negative"""
        if theta_e < self.theta_e_n_thresh:
            return 1.0
        elif theta_e < self.theta_e_z_thresh:
            return (self.theta_e_z_thresh - theta_e) / (self.theta_e_z_thresh - self.theta_e_n_thresh)
        else:
            return 0.0

    def theta_e_z(self, theta_e: float):
        """fuzzy membership function for theta_e zero"""
        if theta_e < self.theta_e_n_thresh:
            return 0.0
        elif theta_e < self.theta_e_z_thresh:
            return (theta_e - self.theta_e_n_thresh) / (self.theta_e_z_thresh - self.theta_e_n_thresh)
        elif theta_e < self.theta_e_p_thresh:
            return (self.theta_e_p_thresh - theta_e) / (self.theta_e_p_thresh - self.theta_e_z_thresh)
        else:
            return 0.0

    def theta_e_p(self, theta_e: float):
        """fuzzy membership function for theta_e positive"""
        if theta_e < self.theta_e_z_thresh:
            return 0.0
        elif theta_e < self.theta_e_p_thresh:
            return (theta_e - self.theta_e_z_thresh) / (self.theta_e_p_thresh - self.theta_e_z_thresh)
        else:
            return 1.0

    def _get_nearest_waypoint(self, x: float, y: float, update_prev_idx: bool = False):
        """search the closest waypoint to the vehicle on the reference path"""
        SEARCH_IDX_LEN = 200 # [points] forward search range
        prev_idx = self.prev_waypoints_idx
        dx = [x - ref_x for ref_x in self.ref_path[prev_idx:(prev_idx + SEARCH_IDX_LEN), 0]]
        dy = [y - ref_y for ref_y in self.ref_path[prev_idx:(prev_idx + SEARCH_IDX_LEN), 1]]
        d = [idx ** 2 + idy ** 2 for (idx, idy) in zip(dx, dy)]
        min_d = min(d)
        nearest_idx = d.index(min_d) + prev_idx

        # get reference values of the nearest waypoint
        ref_x = self.ref_path[nearest_idx,0]
        ref_y = self.ref_path[nearest_idx,1]
        ref_yaw = self.ref_path[nearest_idx,2]
        ref_v = self.ref_path[nearest_idx,3]

        # update nearest waypoint index if necessary
        if update_prev_idx:
            self.prev_waypoints_idx = nearest_idx 

        return nearest_idx, ref_x, ref_y, ref_yaw, ref_v

## Simulation of Path-Tracking
- Longitudinal Control : Fuzzy Controller
- Lateral Control : Fuzzy Controller

In [ ]:
# simulation settings
delta_t = 0.05 # [sec]
sim_steps = 1000 # [steps]
print(f"[INFO] delta_t : {delta_t:.2f}[s] , sim_steps : {sim_steps}[steps], total_sim_time : {delta_t*sim_steps:.2f}[s]")

# load and visualize reference path
ref_path = np.genfromtxt('./ovalpath.csv', delimiter=',', skip_header=1)
plt.title("Reference Path")
plt.plot(ref_path[:,0], ref_path[:,1])
plt.show()

# initialize a vehicle as a control target
vehicle = Vehicle(
    max_steer_abs=0.523, # [rad]
    max_accel_abs=2.000, # [m/s^2]
    ref_path = ref_path[:, 0:2], # ndarray, size is <num_of_waypoints x 2>
)
vehicle.reset(
    init_state = np.array([0.0, 1.0, 0.0, 0.0]), # [x[m], y[m], yaw[rad], v[m/s]]
)
vehicle_trajectory = np.array([vehicle.get_state()[:2]])

# initialize fuzzy controller for acceleration control
fuzzy_lon_controller = FuzzyLongitudinalController(
    target_velocity = +5.0 # [m/s]
)

# initialize fuzzy controller for steering control
fuzzy_lat_controller = FuzzyLateralController(
    ref_path = ref_path, # ndarray, size is <num_of_waypoints x 2>
)

# simulation loop
for i in range(sim_steps):

    # get current state of vehicle
    current_state = vehicle.get_state()

    try:
        # calculate control inputs
        current_velocity = current_state[3]
        accel_input = fuzzy_lon_controller.calc_control_input(observed_vel=current_velocity, delta_t=delta_t)
        steer_input = fuzzy_lat_controller.calc_control_input(observed_x=current_state, delta_t=delta_t)

    except IndexError as ex:
        # the vehicle has reached the end of the reference path
        print("[ERROR] IndexError detected. Terminate simulation.")
        break

    # print current state and input force
    print(f"Time: {i*delta_t:>2.2f}[s], x={current_state[0]:>+3.3f}[m], y={current_state[1]:>+3.3f}[m], yaw={current_state[2]:>+3.3f}[rad], v={current_state[3]:>+3.3f}[m/s], steer={steer_input:>+6.2f}[rad], accel={accel_input:>+6.2f}[m/s]")

    # update states of vehicle
    vehicle.update(u=[steer_input, accel_input], delta_t=delta_t, vehicle_traj=vehicle_trajectory) # update vehicle state
    vehicle_trajectory = np.vstack((vehicle_trajectory, vehicle.get_state()[:2])) # record vehicle trajectory

# show animation
vehicle.show_animation(interval_ms=int(delta_t * 1000))

# save animation as a mp4 file if necessary
# vehicle.save_animation("fuzzy_pathtracking_demo.mp4", interval=int(delta_t * 1000), movie_writer="ffmpeg") # ffmpeg is required to write mp4 file